# Gun violence cases in the United States of Ameria

According to CDC, "One person is killed by a firearms every 17 minutes, 87 people are killed during an average day, and 609 are killed every week”(CDC). Gun violence has become one of the biggest problems in the history of the country. The alarming mortality rate as well as how frequenctly the crime occurs has made it a focus during every election but we have still failed to solve or even address this problem propery.

In [1]:
# !pip install folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import folium
import requests
import random
from folium.map import *
from folium import plugins
from folium.plugins import MeasureControl
from folium.plugins import FloatImage



ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy c-extensions failed.
- Try uninstalling and reinstalling numpy.
- If you have already done that, then:
  1. Check that you expected to use Python3.6 from "/Users/pradyumnbangera/Desktop/final/env/bin/python3",
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy version "1.17.4" you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed numpy
     - your operating system
     - whether or not you have multiple versions of Python installed
     - if you built from source, your compiler versions and ideally a build log

- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.

Note: this error has many pos

# Cleaning data: Removing NaN value from latitude and longitude

In [ ]:
# print(pd.__version__)
data = pd.read_csv("gun-violence-data_01-2013_03-2018.csv")
# print(len(data))
data = data[pd.notnull(data.latitude)]
data = data[pd.notnull(data.longitude)]
# print(len(data))
data.head()

# Plotting the crime data on the map

Here we have just used the latitude and logitude values from every gun violence case and plotted them on the folium graph using circle markers for a good idea about how these gun violence crimes are distributed throughout the entire country.

In [ ]:
from folium.plugins import FastMarkerCluster
map_osm = folium.Map(location=[48, -102], zoom_start=4)
counter = 0
ct= 0 
sam = data.sample(3500)
FastMarkerCluster(data = list(zip(sam['latitude'].values, sam['longitude'].values))).add_to(map_osm)
for index,loc in sam.iterrows():

        if(pd.notnull(loc[14]) and pd.notnull(loc[16])):
            latlong = [loc[14], loc[16]]
        folium.CircleMarker(location=latlong, radius=8, fill_opacity=.5, fill=True).add_to(map_osm)
folium.LayerControl().add_to(map_osm)
    
map_osm

# Cleaning data: Splitting dates for better acessibillity and Plotting total gun violence cases in every year

In the following piece of code, we have just worked on tidying and scraping the data required which in this case is 
splitting month into 3 different columns of month year and day so that we can access it more easily. We have further used 
it to acquire the frequency of gun violence crimes over the different years so that we can get an idea about 
whether we have had a yearlly incline or decline.

In [ ]:
# !pip install seaborn
import collections
import seaborn as sns
year = []
month = []
day = []
q = []
count = 0
uniqueyear = {}
for index,loc in sam.iterrows():
    year.append(int(loc[1].split('-')[0]))
    month.append(int(loc[1].split('-')[1]))
    day.append(int(loc[1].split('-')[2]))
sam['Year'] = year
sam['Month'] = month
sam['Day'] = day

for i in year:
    if i not in q:
        q.append(i)
        uniqueyear[i] = 0
        
for i in year:
    uniqueyear[i] += 1
    
uniqueyear = collections.OrderedDict(sorted(uniqueyear.items()))

f = []
y = []
for i in uniqueyear:
    f.append(uniqueyear[i])
    y.append(i)
p = pd.DataFrame({'Year' : y , 'frequency' : f})
        
plt.figure(figsize = (20, 10))
ax = sns.barplot(x = p['Year'], y = p['frequency'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right",fontsize=20)
plt.tight_layout()
plt.show()

expvyear = p[['Year','frequency']]
x = expvyear['year'].values
y = expvyear['frequency'].values
plt.plot(x, y, 'o')
plt.xlabel("Year", fontsize=10)
plt.ylabel("Frequency", fontsize=10)
plt.show()



# Plotting the total gun cases against every year

In the following plot graph, we have made an effort to plot the respective number of gun violence cases
against their respective years, I have done this by initially creating a dictionary that maps each year to
the total number of gun violence cases that have happened in that respective year. Eventually i have mapped 
every key and and its values to different lists. Furthermore, I have then created a pandas dataframe that just 
has two columns-Year and frequency. Using the dataframe, I have plotted the columns on the X and Y axis 
using Matplotlib to give us a good idea of how variable gun violence has been over the years between 2013
and 2018.

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))



p.plot(x = 'Year', y = 'frequency', ax = ax)
plt.ylabel('Frequency')
ax.set_xlim([2013,2018])
plt.show()

In [ ]:
from mapboxgl.utils import *
from mapboxgl.viz import *

token = os.getenv('MAPBOX_ACCESS_TOKEN')
print(sam['longitude'])
# Create a geojson Feature Collection export from a Pandas dataframe
points = df_to_geojson(sam, properties = None, lat='latitude', lon='longitude', precision=3)

#Create a heatmap
heatmap_color_stops = create_color_stops([0.01,0.25,0.5,0.75,1], colors='RdPu')
heatmap_radius_stops = [[0,1], [15, 40]] #increase radius with zoom

color_breaks = [0,10,100,1000,10000]
color_stops = create_color_stops(color_breaks, colors='Spectral')

heatmap_weight_stops = create_weight_stops(color_breaks)

#Create a heatmap
viz = HeatmapViz(points,
                 access_token=token,
                 weight_property= None,
                 weight_stops=heatmap_weight_stops,
                 color_stops=heatmap_color_stops,
                 radius_stops=heatmap_radius_stops,
                 opacity=0.9,
                 center=(-95, 40),
                 zoom=3,
                 below_layer='waterway-label')

viz.show()

# Gun violence data for each state

In the following dataset, we have tried to depict the total number of gun violence cases that have taken place in every state in the United States of America. We have done so by counting the total occurences of rows in the data frame with the respective states. Furthermore we have created a dictionary to keep track of this data and later used the dictionary to create a data frame with columns - states and Frequency. We have later plotted the data using a bar plot.

In [ ]:
state = []
unst = {}
for i in sam['state']:
    if i not in state:
        state.append(i)
        unst[i] = 0

for i in sam['state']:
    unst[i] += 1
    
unst = collections.OrderedDict(sorted(unst.items()))

s = []
f = []
for i in unst:
    f.append(unst[i])
    s.append(i)
p = pd.DataFrame({'State' : s , 'frequency' : f})

    


plt.figure(figsize = (20, 10))
ax = sns.barplot(x = p['State'], y = p['frequency'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right",fontsize=20)
plt.tight_layout()
plt.show()

# Graph to notice trend of incline/decline in each state

In the following graph, we have attempted to display the total cases of gun vioilence that have taken place inevery state of the United states of America between the period from 2013 - 2018. In order to do so we created a new panda dataframe that condidted of columns - Year, States, and Frequency/count of gun violence crimes. This dataframe allowed us to present total gun violence crimes in each state in each year in a very concise manner. Later we grouped the data frame by the name of the state and plotted it on the graph.

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
# sam = sam.groupby('Year')
count2013 = {}
count2014 = {}
count2015 = {}
count2016 = {}
count2017 = {}
count2018 = {}
for i, tabs in sam:
    for s in state:
#         print(s)
        for r in tabs.iterrows():
            if s == r[1][2]:
                count2013[s] = 0
                count2014[s] = 0
                count2015[s] = 0
                count2016[s] = 0
                count2017[s] = 0
                count2018[s] = 0
                
for i, tabs in sam:
    if i ==2013:
        for s in state:
            for r in tabs.iterrows():
                if s == r[1][2]:
                    count2013[s] += 1
                
    if i ==2014:
        for s in state:
            for r in tabs.iterrows():
                if s == r[1][2]:
                    count2014[s] += 1
    if i ==2015:
        for s in state:
            for r in tabs.iterrows():
                if s == r[1][2]:
                    count2015[s] += 1
    if i ==2016:
        for s in state:
            for r in tabs.iterrows():
                if s == r[1][2]:
                    count2016[s] += 1
    if i ==2017:
        for s in state:
            for r in tabs.iterrows():
                if s == r[1][2]:
                    count2017[s] += 1
                    
    if i ==2018:
        for s in state:
            for r in tabs.iterrows():
                if s == r[1][2]:
                    count2018[s] += 1
                    

y = []
stt = []
freq = []



for i in count2013:
    if count2013[i]>0:
        y.append(2013)
        stt.append(i)
        freq.append(count2013[i])
        
for i in count2014:
    if count2014[i]>0:
        y.append(2014)
        stt.append(i)
        freq.append(count2014[i])
    
for i in count2015:
    if count2015[i]>0:
        y.append(2015)
        stt.append(i)
        freq.append(count2015[i])
        
for i in count2016:
    if count2016[i]>0:
        y.append(2016)
        stt.append(i)
        freq.append(count2016[i])
        
        
for i in count2017:
    if count2017[i]>0:
        y.append(2017)
        stt.append(i)
        freq.append(count2017[i])
        
        
for i in count2018:
    if count2014[i]>0:
        y.append(2018)
        stt.append(i)
        freq.append(count2018[i])
        
    
df = pd.DataFrame({'Year':y, 'State':stt,'Frequency':freq})
    

probtwo = df.groupby('State')
for stateName, tabs in probtwo:
    tabs.plot(x = 'Year', y = 'Frequency', ax = ax, label = stateName)
plt.ylabel('Frequency')
plt.xlabel('Year')
ax.set_xlim([2013,2018])
plt.show()